In [1]:
from gnn_tracking.utils.loading import TrackingDataModule
from pytorch_lightning import Trainer
from gnn_tracking.training.tc import TCModule
from pathlib import Path
from rich.pretty import pprint
from pytorch_lightning.utilities.model_summary import ModelSummary

In [2]:
dm = TrackingDataModule(
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v6/part_1/"
        ],
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v6/part_9/"
        ],
        stop=5,
    ),
)

In [3]:
# Add path to model exchange
model_exchange_path = Path("/home/kl5675/Documents/23/git_sync/model-exchange/models")
assert model_exchange_path.exists(), model_exchange_path

In [4]:
lmodel = TCModule.load_from_checkpoint(model_exchange_path / "tc-04b2e3ce.ckpt")

[14:32:48] DEBUG: Getting class PreTrainedECGraphTCN from module gnn_tracking.models.track_condensation_networks
[14:32:48] DEBUG: Getting class ECForGraphTCN from module gnn_tracking.models.edge_classifier
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'ec' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['ec'])`.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.
  rank_zero_warn(
[14:32:48] DEBUG: Getting class MLGraphConstruction from module gnn_tracking.models.graph_construction
[14:32:48] DEBUG: Getting class GraphConstructionFCN

In [27]:
ModelSummary(lmodel)

  | Name            | Type                 | Params
---------------------------------------------------------
0 | model           | PreTrainedECGraphTCN | 1.8 M 
1 | preproc         | MLGraphConstruction  | 1.3 M 
2 | potential_loss  | PotentialLoss        | 0     
3 | background_loss | BackgroundLoss       | 0     
---------------------------------------------------------
1.8 M     Trainable params
1.3 M     Non-trainable params
3.2 M     Total params
12.636    Total estimated model params size (MB)

In [28]:
pprint(
    dict(lmodel.hparams),
)

{
│   'model': {
│   │   'class_path': 'gnn_tracking.models.track_condensation_networks.PreTrainedECGraphTCN',
│   │   'init_args': {
│   │   │   'node_indim': 22,
│   │   │   'edge_indim': 44,
│   │   │   'h_dim': 192,
│   │   │   'e_dim': 192,
│   │   │   'h_outdim': 12,
│   │   │   'hidden_dim': 192,
│   │   │   'L_hc': 3,
│   │   │   'alpha_hc': 0.5,
│   │   │   'feed_edge_weights': True,
│   │   │   'ec_threshold': 0.2,
│   │   │   'mask_orphan_nodes': False,
│   │   │   'use_ec_embeddings_for_hc': True,
│   │   │   'ec': {
│   │   │   │   'class_path': 'gnn_tracking.models.edge_classifier.ECForGraphTCN',
│   │   │   │   'init_args': {
│   │   │   │   │   'node_indim': 22,
│   │   │   │   │   'edge_indim': 44,
│   │   │   │   │   'interaction_node_dim': 128,
│   │   │   │   │   'interaction_edge_dim': 128,
│   │   │   │   │   'hidden_dim': 128,
│   │   │   │   │   'L_ec': 3,
│   │   │   │   │   'alpha': 0.35,
│   │   │   │   │   'residual_type': 'skip1',
│   │   │   │   │   'use_intermediate_edge_embeddings': False,
│   │   │   │   │   'use_node_embedding': True,
│   │   │   │   │   'residual_kwargs': None
│   │   │   │   }
│   │   │   }
│   │   }
│   },
│   'preproc': {
│   │   'class_path': 'gnn_tracking.models.graph_construction.MLGraphConstruction',
│   │   'init_args': {
│   │   │   'max_radius': 0.8,
│   │   │   'max_num_neighbors': 64,
│   │   │   'use_embedding_features': True,
│   │   │   'ratio_of_false': None,
│   │   │   'build_edge_features': True,
│   │   │   'ec_threshold': None,
│   │   │   'ml_freeze': True,
│   │   │   'ec_freeze': True,
│   │   │   'ml': {
│   │   │   │   'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN',
│   │   │   │   'init_args': {'in_dim': 14, 'hidden_dim': 512, 'out_dim': 8, 'depth': 6, 'beta': 0.4}
│   │   │   },
│   │   │   'ec': None
│   │   }
│   },
│   'lw_repulsive': 0.743380428762342,
│   'lw_background': 0.0041,
│   'potential_loss': {
│   │   'class_path': 'gnn_tracking.metrics.losses.PotentialLoss',
│   │   'init_args': {'q_min': 0.01, 'radius_threshold': 1.0, 'attr_pt_thld': 0.9}
│   },
│   'background_loss': {'class_path': 'gnn_tracking.metrics.losses.BackgroundLoss', 'init_args': {'sb': 0.1}},
│   'cluster_scanner': {
│   │   'class_path': 'gnn_tracking.postprocessing.dbscanscanner.DBSCANHyperParamScanner',
│   │   'init_args': {
│   │   │   'eps_range': (1e-05, 1.0),
│   │   │   'min_samples_range': (1, 1),
│   │   │   'n_jobs': 6,
│   │   │   'guide': 'trk.double_majority_pt0.9',
│   │   │   'n_trials': 20
│   │   }
│   }
}

In [29]:
trainer = Trainer(accelerator="gpu")

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [30]:
trainer.validate(lmodel, dm, ckpt_path="last")

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:189: UserWarning: .validate(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
[14:42:47] INFO: DataLoader will load 5 graphs (out of 1000 available).
[14:42:47] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v6/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v6/part_9/data29004_s0.pt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/scratch/gpf

Validation: 0it [00:00, ?it/s]

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf fou

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃            Validate metric             ┃              DataLoader 0              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│               attractive               │         1.3140040209691506e-05         │
│             attractive_std             │         2.981064710638748e-07          │
│          attractive_weighted           │         1.3140040209691506e-05         │
│        attractive_weighted_std         │         2.981064710638748e-07          │
│               background               │                  1.0                   │
│             background_std             │         1.332800358966324e-08          │
│          background_weighted           │          0.00409999955445528           │
│        background_weighted_std         │         1.0412502804424406e-10         │
│            best_dbscan_eps             │          0.39528666512149513           │
│          best_dbscan_eps_std           │                  nan                   │
│        best_dbscan_min_samples         │                  1.0                   │
│      best_dbscan_min_samples_std       │                  nan                   │
│               repulsive                │         2.9917075153207406e-05         │
│             repulsive_std              │         1.1564700344024459e-06         │
│           repulsive_weighted           │         2.223976844106801e-05          │
│         repulsive_weighted_std         │          8.59697195210174e-07          │
│                 total                  │          0.004135379567742348          │
│               total_std                │         1.1014168421752402e-06         │
│          trk.double_majority           │           0.2574138641357422           │
│       trk.double_majority_pt0.5        │           0.5385388731956482           │
│     trk.double_majority_pt0.5_std      │          0.012896901927888393          │
│   trk.double_majority_pt0.5_std_std    │                  nan                   │
│       trk.double_majority_pt0.9        │           0.7287040948867798           │
│     trk.double_majority_pt0.9_std      │          0.014321673661470413          │
│   trk.double_majority_pt0.9_std_std    │                  nan                   │
│       trk.double_majority_pt1.5        │           0.740193784236908            │
│     trk.double_majority_pt1.5_std      │          0.014862671494483948          │
│   trk.double_majority_pt1.5_std_std    │                  nan                   │
│        trk.double_majority_std         │          0.006116211414337158          │
│      trk.double_majority_std_std       │                  nan                   │
│        trk.fake_double_majority        │          0.06842746585607529           │
│     trk.fake_double_majority_pt0.5     │          0.06121145933866501           │
│   trk.fake_double_majority_pt0.5_std   │         0.0028727513272315264          │
│ trk.fake_double_majority_pt0.5_std_std │                  nan                   │
│     trk.fake_double_majority_pt0.9     │          0.051622968167066574          │
│   trk.fake_double_majority_pt0.9_std   │          0.005237561650574207          │
│ trk.fake_double_majority_pt0.9_std_std │                  nan                   │
│     trk.fake_double_majority_pt1.5     │          0.054189544171094894          │
│   trk.fake_double_majority_pt1.5_std   │          0.006918206345289946          │
│ trk.fake_double_majority_pt1.5_std_std │                  nan                   │
│      trk.fake_double_majority_std      │          0.003320294199511409          │
│    trk.fake_double_majority_std_std    │                  nan                   │
│              trk.fake_lhc              │          0.24259431660175323           │
│           trk.fake_lhc_pt0.5           │          0.16434533894062042           │
│         trk.fake_lhc_pt0.5_std         │          

[{'attractive': 1.3140040209691506e-05,
  'repulsive': 2.9917075153207406e-05,
  'background': 1.0,
  'attractive_weighted': 1.3140040209691506e-05,
  'repulsive_weighted': 2.223976844106801e-05,
  'background_weighted': 0.00409999955445528,
  'total': 0.004135379567742348,
  'trk.i_batch': 2.0,
  'trk.n_particles': 1115.4000244140625,
  'trk.n_cleaned_clusters': 362.6000061035156,
  'trk.perfect': 0.12285707890987396,
  'trk.double_majority': 0.2574138641357422,
  'trk.lhc': 0.757405698299408,
  'trk.fake_perfect': 0.2029842585325241,
  'trk.fake_double_majority': 0.06842746585607529,
  'trk.fake_lhc': 0.24259431660175323,
  'trk.n_particles_pt0.5': 487.0,
  'trk.n_cleaned_clusters_pt0.5': 290.6000061035156,
  'trk.perfect_pt0.5': 0.2747100591659546,
  'trk.double_majority_pt0.5': 0.5385388731956482,
  'trk.lhc_pt0.5': 0.8356546759605408,
  'trk.fake_perfect_pt0.5': 0.3250402808189392,
  'trk.fake_double_majority_pt0.5': 0.06121145933866501,
  'trk.fake_lhc_pt0.5': 0.16434533894062042